In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier

# Bibliotecas Locais
from carga import *
from tratamentos import *
from categorizacao import *

pfEnade = doCarga('microdados_enade_2019.txt')
# TRATAMENTOS 
pfEnade = doIdade(pfEnade)
pfEnade = doNota(pfEnade)
pfEnade = doVazios(pfEnade)
pfEnade = doPublicoPrivada(pfEnade)
pfEnade = doHumanasExatas(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doMigracao(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doEscolaridade(pfEnade)
pfEnade = doCategoricos(pfEnade)

X, y, x_train, x_test, y_train, y_test = doSplit(pfEnade)

# Bagging

In [3]:
# Decision Tree
bg = BaggingClassifier(DecisionTreeClassifier(), max_samples= 0.5, max_features = 1.0, n_estimators = 20)
bg.fit(x_train, y_train.values.ravel())
bg.score(x_test,y_test)

0.6454416524237245

In [6]:
# kNN
bg = BaggingClassifier(KNeighborsClassifier(n_neighbors=30), max_samples= 0.5, max_features = 1.0, n_estimators = 20)
bg.fit(x_train, y_train.values.ravel())
bg.score(x_test,y_test)

0.6619513436852249

In [7]:
# Randon Forest
bg = BaggingClassifier(RandomForestClassifier(n_estimators=20), max_samples= 0.5, max_features = 1.0, n_estimators = 20)
bg.fit(x_train, y_train.values.ravel())
bg.score(x_test,y_test)

0.6523873595403469

In [10]:
# SVN
from sklearn import svm
bg = BaggingClassifier(svm.SVC(kernel='linear', C=1.0), max_samples= 0.5, max_features = 1.0, n_estimators = 20)
bg.fit(x_train, y_train.values.ravel())
bg.score(x_test,y_test)

0.6176588239572348

# ADA BOOST

In [11]:
# Decision Tree
adb = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators = 5, learning_rate = 1)
adb.fit(x_train, y_train.values.ravel())
adb.score(x_test, y_test)

0.6464598712680462

In [14]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
adb = AdaBoostClassifier(KNeighborsClassifier(n_neighbors=30), n_estimators = 5, learning_rate = 1)
adb.fit(x_train, y_train.values.ravel())
adb.score(x_test, y_test)

ValueError: KNeighborsClassifier doesn't support sample_weight.

In [15]:
# Randon Forest
adb = AdaBoostClassifier(RandomForestClassifier(n_estimators=20), n_estimators = 5, learning_rate = 1)
adb.fit(x_train, y_train.values.ravel())
adb.score(x_test, y_test)

0.6451507327539183

In [22]:
# SVM
from sklearn import svm
adb = AdaBoostClassifier(svm.SVC(kernel='linear', C=1.0), algorithm='SAMME',  n_estimators = 5, learning_rate = 1)
adb.fit(x_train, y_train.values.ravel())
adb.score(x_test, y_test)

0.5390014182333903

# Voting Classifier

In [23]:
# Voting Classifier - Multiple Model Ensemble 
l#r = LogisticRegression()
dt
 #Decision Tree

dtclas = DecisionTreeClassifier()k#NN
R
aknnclas= KNeighborsClassifier(n_neighbors=30)#ndon Forest
SV
Nrfc las= RandomForestClassifier()#= V
otingClassifier( estimators= [('lr',lr),('dt'dtclast)dtclas'rfcknnclasrfknnclas], vrfclasngrfclas'hard')
evc.fit(x_train, y_train.values.ravel())
evc.score(x_test, y_test)

0.6496963525946398

# Hiperparametros

In [29]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rf = RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random.fit(x_train, y_train.values.ravel())
rf_random.score(x_test, y_test.values)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 18.4min finished


0.6627877377359177

In [30]:
# Faz teste com melhor resultado
best_random = rf_random.best_estimator_
best_random.fit(x_train, y_train.values.ravel())
best_random.score(x_test, y_test)

0.6634423069929816

In [31]:
rf_random.best_estimator_

RandomForestClassifier(max_depth=10, max_features='sqrt', min_samples_leaf=4,
                       n_estimators=1560)

In [32]:
# OUTRA RODADA RESTRINGINDO ENTRE OS MELHORADOS
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
#rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)

In [34]:
# Fit the grid search to the data
grid_search.fit(x_test, y_test)
grid_search.best_params_

Fitting 3 folds for each of 288 candidates, totalling 864 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  6.0min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  8.5min finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 90,
 'max_features': 2,
 'min_samples_leaf': 5,
 'min_samples_split': 12,
 'n_estimators': 100}

In [36]:
# best_grid.fit(X_train, y_train.values.ravel())
grid_search.score(x_test, y_test)

0.7035528564675079